<a href="https://colab.research.google.com/github/acoiman/pdt/blob/main/asthma_mortality/notebooks/colab/08_2Asthma_Mortality_RF_RPIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Asthma Mortality in Argentina using Remote Sensing Data and Machine Learning



In this notebook, we will use remote sensing data and Random Forest (RF) to predict asthma mortality in Argentina at departmental level from 2001 to 2022. We will model the Normalized Asthma Mortality Rate (NAMR) in a two-stage RF approach—classification followed by regression—using predictor variables derived from satellite-based observations such as burned areas, and Particulate Matter with 2.5 micrometers in diameter or less (PM2.5), along with Population Density (PD), and lagged and feature engineered variables.

Import required libraries

In [ ]:
# dataframe libraries
import pandas as pd
import numpy as np

# geospatial libraries
import geopandas as gpd
import mapclassify
from libpysal.weights import Queen
from esda.moran import Moran
from pysal.explore import esda

# plot libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from matplotlib.patches import Patch
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# modelling libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score,explained_variance_score,median_absolute_error, max_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# other libraries
from joblib import Parallel, delayed
import shap
from itables import init_notebook_mode, show

## Load and reduce data

In [ ]:
%cd work/

In [ ]:
# Load dataset with data per department
gdf = gpd.read_file("pdt/asthma_mortality/data/gpkg/tma_pm25_ba_pd_pdpm25_2001_2022.gpkg")

In [ ]:
# Drop geometry if only panel analysis is needed
df = gdf.drop(columns="geometry")

In [ ]:
# Reshape df to long format
years = range(2001, 2023)
records = []

In [ ]:
for _, row in df.iterrows():
    iddpto = row["IDDPTO"]
    for year in years:
        records.append({
            "IDDPTO": iddpto,
            "YEAR": year,
            "CA": row.get(f"CA_{year}", np.nan),
            "PM25": row.get(f"PM25_{year}", np.nan),
            "NBA": row.get(f"NBA_{year}", np.nan),
            "PD": row.get(f"PD_{year}", np.nan),
            "PDPM25": row.get(f"PDPM25_{year}", np.nan)
            })

In [ ]:
# create new df from list and sort
panel_df = pd.DataFrame(records)

In [ ]:
# Sort and reset index
panel_df = panel_df.sort_values(by=["IDDPTO", "YEAR"]).reset_index(drop=True)

## Exploratory Data Analysis (EDA)

In [ ]:
# Create a copy of the panel_df DataFrame for exploratory data analysis (EDA)
df_eda = panel_df.copy()

In [ ]:
# Convert the 'YEAR' column in panel_df to datetime format and assign it to df_eda
df_eda['YEAR'] = pd.to_datetime(panel_df['YEAR'], format='%Y')

In [ ]:
# visualize the first rows
init_notebook_mode(all_interactive=True)
show(df_eda)

In [ ]:
# get the number of rows
len(df_eda)

In [ ]:
# Filter the year 2022 (Test set)
df_eda_2022 = df_eda[df_eda['YEAR'] == pd.to_datetime(2022, format='%Y')]

In [ ]:
# visualize the first rows
init_notebook_mode(all_interactive=False)
df_eda_2022.CA.describe()

In [ ]:
# create a box plot for CA column
sns.boxplot(x=df_eda_2022['CA'])
plt.show()

In [ ]:
# create a barplot showing the distribution  pf CA column
sns.histplot(df_eda_2022['CA'], bins=25)
plt.xlabel('CA')
plt.ylabel('Frequency')
plt.title('Distribution of NAMR')
plt.xlabel("NAMR")
plt.show()

In [ ]:
# how many zero values are there for CA column
df_eda_2022[df_eda_2022['CA'] == 0].shape[0]

In [ ]:
# in percentage
round(((df_eda_2022[df_eda_2022['CA'] == 0].shape[0] / len(df_eda_2022)) * 100), 2)

In [ ]:
# Filter out the year 2022
df_eda_0121 = df_eda[df_eda['YEAR'] < pd.to_datetime(2022, format='%Y')]

In [ ]:
# Set the 'YEAR' column as the index for the DataFrame
df_eda_0121.set_index('YEAR', inplace=True)

In [ ]:
# Plot mortality rate over time
plt.figure(figsize=(6, 4))
sns.lineplot(data=df_eda_0121, x=df_eda_0121.index, y='CA', marker='o', estimator='mean')
plt.title('Mean Mortality Rate Over Time')
plt.ylabel('Mean NAMR')
plt.xlabel('Year')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# ACF and PACF plots
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

plot_pacf(df_eda_0121['CA'], ax=axes[0], lags=21, title='Partial Autocorrelation (PACF)')
plot_acf(df_eda_0121['CA'], ax=axes[1], lags=21, title='Autocorrelation (ACF)')

plt.tight_layout()
plt.show()

### Exploratory Spatial  Data Analysis (ESDA)

In [ ]:
# Load dataset with data per department
gdf = gpd.read_file("pdt/asthma_mortality/data/gpkg/tma_pm25_ba_pd_pdpm25_2001_2022.gpkg")

In [ ]:
# Reshape gdf to long format
years = range(2001, 2023)
records = []

In [ ]:
for _, row in gdf.iterrows():
    iddpto = row["IDDPTO"]
    geometry = row["geometry"]
    for year in years:
        records.append({
            "IDDPTO": iddpto,
            "YEAR": year,
            "CA": row.get(f"CA_{year}", np.nan),
            "PM25": row.get(f"PM25_{year}", np.nan),
            "NBA": row.get(f"NBA_{year}", np.nan),
            "PD": row.get(f"PD_{year}", np.nan),
            "PDPM25": row.get(f"PDPM25_{year}", np.nan),
            "geometry": geometry # Add geometry
            })

In [ ]:
# create new df from list and sort
panel_gdf = pd.DataFrame(records)

In [ ]:
# Sort and reset index
panel_gdf = panel_gdf.sort_values(by=["IDDPTO", "YEAR"]).reset_index(drop=True)

In [ ]:
# visualize the fisrt rows
init_notebook_mode(all_interactive=True)
show(panel_gdf)

In [ ]:
# Filter the DataFrame to include rows where 'YEAR' is less than 2022 and create a copy
gdf_esda_0121 = panel_gdf[panel_gdf['YEAR'] < 2022].copy()

In [ ]:
# Define spatial weights matrix using Queen contiguity
w = Queen.from_dataframe(gdf_esda_0121)
w.transform = 'R'

In [ ]:
# Compute Moran's I for each mean feature
moran_results = {}
for col in ['CA', 'PM25', 'NBA', 'PD', 'PDPM25']:
    moran = Moran(gdf_esda_0121[col], w)
    moran_results[col] = {'Moran_I': moran.I, 'Moran_p_sim': moran.p_sim}
# Create a table (DataFrame) from the results
moran_df = pd.DataFrame.from_dict(moran_results, orient='index')

In [ ]:
# Display the table
init_notebook_mode(all_interactive=True)
show(moran_df)

## Part 1 – Classification Model

In this section we will  train and evaluate a classification model to predict whether NAMR is classified as an absence (0) or a presence (1) of asthma mortality (binary classification)

In [ ]:
# Create a copy of the DataFrame to preserve the original data
df_ts = panel_df.copy()

In [ ]:
# Create lag variables (up to 2 years)
def create_lags(df, var, max_lag=2):
    for lag in range(1, max_lag+1):
        df[f"{var}_lag{lag}"] = df[var].shift(lag)
    return df

In [ ]:
for var in ["PM25", "NBA", "PD", "PDPM25"]:
    df_ts = create_lags(df_ts, var)

In [ ]:
for var in ["CA"]:
    df_ts = create_lags(df_ts, var,  max_lag=4)

In [ ]:
# Drop the initial rows with NaNs due to lagging
df_ts = df_ts.dropna().reset_index(drop=True)

In [ ]:
# Binary target
df_ts['CA_bin'] = (df_ts['CA'] > 0).astype(int)

In [ ]:
# Split train and test
train = df_ts[df_ts['YEAR'] < 2022].copy()
test = df_ts[df_ts['YEAR'] == 2022].copy()

In [ ]:
# visualize the first rows
init_notebook_mode(all_interactive=True)
show(train)

In [ ]:
# Define the list of predictor variables for the model
predictors = ['PM25', 'NBA', 'PD', 'PDPM25',
              'PM25_lag1', 'PM25_lag2', 'NBA_lag1','NBA_lag2',
              'PD_lag1', 'PD_lag2', 'PDPM25_lag1','PDPM25_lag2',
              'CA_lag1', 'CA_lag2', 'CA_lag3','CA_lag4']

In [ ]:
# Create a RandomForestClassifier with a fixed random state for reproducibility and fit it to the training data
clf = RandomForestClassifier(random_state=42,
                             n_estimators=100)
clf.fit(train[predictors], train['CA_bin'])

In [ ]:
# Predict probability of CA > 0 in 2022
test['CA_bin_pred'] = clf.predict(test[predictors])

In [ ]:
# visualize the first rows
init_notebook_mode(all_interactive=True)
show(test)

In [ ]:
# Calculate and evaluate classification metrics for the model's predictions
y_true = test['CA_bin']        # Ground truth: binary target variable
y_pred = test["CA_bin_pred"]   # Model's predicted binary values

# Compute accuracy score
acc = accuracy_score(y_true, y_pred)  # Proportion of correct predictions

# Compute precision score
prec = precision_score(y_true, y_pred)  # Proportion of true positives among predicted positives

# Compute recall score
rec = recall_score(y_true, y_pred)  # Proportion of true positives among actual positives

# Compute F1 score
f1 = f1_score(y_true, y_pred)  # Harmonic mean of precision and recall

# Print the calculated metrics with formatting
print(f"Accuracy  = {acc:.3f}")   # Display accuracy with 3 decimal places
print(f"Precision = {prec:.3f}")  # Display precision with 3 decimal places
print(f"Recall    = {rec:.3f}")   # Display recall with 3 decimal places
print(f"F1 Score  = {f1:.3f}")    # Display F1 score with 3 decimal places

Metric	Value	Interpretation
* Accuracy	0.75.	Overall, the classifier correctly identified whether CA was 0 or >0 about 75% of the time. This is decent but may be misleading if there is class imbalance (e.g., many zeros).
* Precision	0.677.	Of all the cases where the classifier predicted CA > 0, 67% were correct. Moderate precision means a fair amount of false positives (it sometimes predicts CA when the true value is 0).
* Recall	0.518.	The model only identified 51.8% of true CA > 0 cases. So it's missing nearly half of the true positives (false negatives are high).
* F1 Score	0.587.	The harmonic mean of precision and recall. This moderate value indicates a trade-off between missing positives and over-predicting them.

##Part 2 – Regression Model

In this section, we will train and evaluate a RF regression model to estimate NAMR values where it is present

In [ ]:
# Train only on CA > 0
train_pos = train[train['CA'] > 0].copy()

In [ ]:
# visualize the first rows
init_notebook_mode(all_interactive=True)
show(train_pos)

In [ ]:
# Train a Random Forest Regressor model with 100 estimators and a fixed random state for reproducibility
reg = RandomForestRegressor(n_estimators=100, random_state=42)
reg.fit(train_pos[predictors], train_pos['CA'])

In [ ]:
# Predict only for departments where classifier said CA > 0
test['CA_pred'] = 0.0  # Initialize the 'CA_pred' column with a default value of 0.0
mask = (test['CA_bin_pred'] == 1) | (test['CA_bin'] == 1)  # Create a mask for rows where either 'CA_bin_pred' or 'CA_bin' equals 1
test.loc[mask, 'CA_pred'] = reg.predict(test.loc[mask, predictors])  # Use the model to predict 'CA_pred' for rows matching the mask

In [ ]:
# Extract the true values from the test dataset for the 'CA' column
true = test['CA']

# Extract the predicted values from the test dataset for the 'CA_pred' column
pred = test['CA_pred']

# Calculate Mean Absolute Error (MAE) between true and predicted values
mae = mean_absolute_error(true, pred)

# Calculate Root Mean Squared Error (RMSE) between true and predicted values
rmse = np.sqrt(mean_squared_error(true, pred))

# Calculate Mean Absolute Percentage Error (MAPE) between true and predicted values
# Adding a small constant (1e-6) to avoid division by zero
mape = np.mean(np.abs((true - pred) / (true + 1e-6))) * 100

# Define a function to calculate Symmetric Mean Absolute Percentage Error (SMAPE)
def smape(A, F):
    # SMAPE formula: 100 * mean(2 * abs(F - A) / (abs(A) + abs(F) + small constant))
    return 100 * np.mean(2 * np.abs(F - A) / (np.abs(A) + np.abs(F) + 1e-6))

# Calculate SMAPE value using the defined function
smape_value = smape(true.values, pred.values)

# Calculate R-squared (R2) score to measure the goodness of fit
r2 = r2_score(true, pred)

# Calculate Explained Variance Score (EVS) to measure the proportion of variance explained
evs = explained_variance_score(true, pred)

# Calculate Median Absolute Error (MedAE) between true and predicted values
medae = median_absolute_error(true, pred)

# Calculate Maximum Error (Max Error) between true and predicted values
max_err = max_error(true, pred)

# Print results
print(f"MAE    = {mae:.3f}")
print(f"RMSE   = {rmse:.3f}")
print(f"MAPE   = {mape:.2f}%")
print(f"SMAPE  = {smape_value:.2f}%")
print(f"R²     = {r2:.3f}")
print(f"Explained Variance = {evs:.3f}")
print(f"Median Absolute Error = {medae:.3f}")
print(f"Max Error = {max_err:.3f}")

In [ ]:
# visualize the first rows
init_notebook_mode(all_interactive=True)
show(test)

## SHAP (SHapley Additive Explanations)

In this section we will interpret the contribution of each independent variable to the final predition using the SHAP method

In [ ]:
# ---- SHAP for Binary Classifier ----
# Use TreeExplainer (optimized for RandomForest)
explainer = shap.TreeExplainer(clf)

In [ ]:
# Sample the test set or use the full set if not too large
X = test[predictors]  # e.g., X = test[predictors].sample(n=200)

In [ ]:
# Function to compute SHAP values for a single row
def compute_shap(row):
    return explainer.shap_values(row)

In [ ]:
# Parallel computation
shap_values_list = Parallel(n_jobs=-1)(
    delayed(compute_shap)(X.iloc[[i]]) for i in range(len(X))
)

In [ ]:
# Combine SHAP values for Class 1 into a single array
shap_values = np.vstack([vals[0][:, 1] for vals in shap_values_list])  # Class 1 SHAPs


In [ ]:
# Generate a SHAP summary plot to visualize the impact of features on the model's predictions
shap.summary_plot(shap_values, X, feature_names=predictors)


In [ ]:
# ---- SHAP for Regression ----
# Use TreeExplainer (optimized for RandomForest)
explainer = shap.TreeExplainer(reg)

# Sample the test set or use the full set if not too large
X = test[predictors]  # e.g., X = test[predictors].sample(n=200)

# Function to compute SHAP values for a single row
def compute_shap(row):
    # For regression, shap_values returns a single 1D array
    return explainer.shap_values(row)

# Parallel computation (adjust n_jobs as needed)
shap_values_list = Parallel(n_jobs=-1)(
    delayed(compute_shap)(X.iloc[[i]]) for i in range(len(X))
)

# Stack the 1D SHAP value arrays vertically
# No need to index [:, 1] as there's only one set of SHAP values for regression
shap_values = np.vstack(shap_values_list)

In [ ]:
# Generate a SHAP summary plot to visualize the impact of features on the model's predictions
shap.summary_plot(shap_values, X, feature_names=predictors)

In [ ]:
# for test change column names CA_pred to CA_2022_PRED
test.rename(columns={'CA_pred': 'CA_2022_PRED'}, inplace=True)

In [ ]:
# save test as a csv file
test.to_csv('pdt/asthma_mortality/data/csv/results_RF.csv', index=False)

## 🌍 Mapping actual vs predicted asthma mortality rate

In [ ]:
# open df with results of RF modelling
results_df = pd.read_csv('pdt/asthma_mortality/data/csv/results_RF.csv', dtype={'IDDPTO': str})

In [ ]:
# Drop columns not required columns in df_results for mapping
df_map = results_df.drop(columns=['YEAR', 'CA', 'PM25', 'NBA', 'PD', 'PDPM25', 'PM25_lag1',
                                  'PM25_lag2', 'NBA_lag1', 'NBA_lag2', 'PD_lag1', 'PD_lag2',
                                  'PDPM25_lag1', 'PDPM25_lag2', 'CA_bin', 'CA_bin_pred',
                                  'CA_lag1', 'CA_lag2', 'CA_lag3','CA_lag4'])

In [ ]:
# round CA_2022_PRED to two decimal places
df_map['CA_2022_PRED'] = df_map['CA_2022_PRED'].round(2)


In [ ]:
# visualize df_map
init_notebook_mode(all_interactive=True)
show(df_map)

In [ ]:
# get df basic info
df_map.info()

In [ ]:
# Join gdf with df_map preserving data of gdf

# Ensure 'IDDPTO' column is of the same type in both dataframes before joining
gdf['IDDPTO'] = gdf['IDDPTO'].astype(str)
df_map['IDDPTO'] = df_map['IDDPTO'].astype(str)

# Perform a left merge (preserving all rows from gdf)
gdf2 = gdf.merge(df_map, on='IDDPTO', how='left')

In [ ]:
# Display info and the first few rows of the merged GeoDataFrame
init_notebook_mode(all_interactive=True)
show(gdf2.head())

In [ ]:
# get df basic info
gdf2.info()

In [ ]:
# save df2 as gpkg file
gdf2.to_file("pdt/asthma_mortality/data/gpkg/results_RF.gpkg", driver="GPKG")

### Calculate classification schema for mapping

We will use [Pysal](https://pysal.org/)'s [mapclassify](https://pysal.org/mapclassify/index.html) library to determine the best classifier for the choropleth map.

We will use the map classifier with the best ACDM (mean Absolute Deviation Around the class Median). In Pysal, ACDM refers to the mean absolute deviation around the class median. It is a measure of a classifier's fit to the data, specifically by evaluating the average distance between each data point and the median value of the assigned class.

In [ ]:
# from gdf2 preserve only IDDPTO, CA_2022 and CA_2022_PRED
df_cl = gdf2[['IDDPTO', 'CA_2022', 'CA_2022_PRED', "geometry"]].copy()

In [ ]:
# visualize the dataframe
init_notebook_mode(all_interactive=True)
show(df_cl.head())

In [ ]:
# get df basic info
df_cl.info()

In [ ]:
# Get the length of the dataframe 'df_cl'
len(df_cl)

In [ ]:
# Select data to analize
selected_data = df_cl.loc[:,["CA_2022", "CA_2022_PRED"]]

In [ ]:
# Classify the data into 4 quantile groups
q4 = mapclassify.Quantiles(selected_data, k=4)
q4

In [ ]:
# Equal Interval Classification
ei5 = mapclassify.EqualInterval(selected_data, k=5)
ei5

In [ ]:
# Classify the data into groups based on the head/tail breaks algorithm
ht = mapclassify.HeadTailBreaks(selected_data)
ht

In [ ]:
# MaximumBreaks classification method
mb5 = mapclassify.MaximumBreaks(selected_data, k=5)
mb5

In [ ]:
# Apply the Standard Deviation and Mean classification method to the selected data.
msd = mapclassify.StdMean(selected_data)
msd

In [ ]:
# Apply Fisher-Jenks classification with 5 classes
fj5 = mapclassify.FisherJenks(selected_data, k=5)
fj5

ACDM(mean Absolute Deviation Around the class Median) visualization

In [ ]:
# Bunch classifier objects
class5 = q4, ei5, ht, mb5, msd, fj5
# Collect ADCM for each classifier
fits = np.array([c.adcm for c in class5])
# Convert ADCM scores to a DataFrame
adcms = pd.DataFrame(fits)
# Add classifier names
adcms["classifier"] = [c.name for c in class5]
# Add column names to the ADCM
adcms.columns = ["ADCM", "Classifier"]
ax = sns.barplot(
    y="Classifier", x="ADCM", data=adcms, hue= adcms["Classifier"],  legend=False
)

### Create choropleth maps

Two classifiers have the lowest ACDM: FisherJenks and HeadTailBreaks. We'll select FisherJenks as the classifier to create the choropleth maps.

In [ ]:
# Convert the bins to a list for further processing
bins = fj5.bins.tolist()
bins

In [ ]:
# insert 0 at 0 position
bins.insert(0, 0.0)
bins

In [ ]:
# Create a custom classification using UserDefined for actual values
classi_actual = mapclassify.UserDefined(df_cl["CA_2022"], bins)

In [ ]:
# Create a custom classification using UserDefined for predicted values
classi_pred = mapclassify.UserDefined(df_cl["CA_2022_PRED"], bins)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 8))
fig.subplots_adjust(hspace=0, wspace=-0.9)
plt.suptitle('Normalized Asthma Mortality Rate 2022', fontsize=14, y=1)

# Plot classi_actual
classi_actual.plot(
    df_cl,
    legend=False,  # We'll build it manually
    axis_on=False,
    border_color='black',
    cmap="viridis_r",
    ax=axes[0]
)

# Plot classi_pred
classi_pred.plot(
    df_cl,
    legend=False,  # We'll build it manually
    axis_on=False,
    border_color='black',
    cmap="viridis_r",
    ax=axes[1]
)


# Custom bin labels and colors
bin_labels = ["0.00", "0.00-0.65", "0.65-2.16", "2.16-4.13", "4.13-7.49", "7.49-14.15"]
n_bins = len(bin_labels)
# cmap = mpl.cm.get_cmap("viridis_r", n_bins)
cmap = mpl.colormaps.get_cmap("viridis_r").resampled(n_bins)
colors = [mpl.colors.to_hex(cmap(i)) for i in range(cmap.N)]

# Create legend patches for bins
bin_patches = [Patch(facecolor=color, edgecolor='black', label=label)
               for color, label in zip(colors, bin_labels)]


# Combine all patches
all_patches = bin_patches

# Display custom legend
#axes[0].legend(handles=all_patches, loc='upper right', bbox_to_anchor=(1.1, 0.4), fontsize=8)
axes[1].legend(handles=all_patches, loc='upper right', bbox_to_anchor=(0.9, 0.25), fontsize=10)

# Set titles
axes[0].set_title('Actual', fontsize=12)
axes[1].set_title('Predicted', fontsize=12)

plt.tight_layout()
plt.show();